In [0]:
sourcesilverCSVFile = "abfss://youtube-silver@practhirusa.dfs.core.windows.net/trending/year=2025/month=09/day=15/silverdata_20250914.csv"

In [0]:
Target_GoldFile = "abfss://youtube-gold@practhirusa.dfs.core.windows.net/trending/year=2025/month=09/day=15/golddata_20250914.csv"

In [0]:
sourceCSV_silverDF = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .option("escape", "\"") \
    .option("multiLine", "true") \
    .option("fs.azure.account.auth.type", "SAS") \
    .option("fs.azure.sas.token.provider.type", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider") \
    .option("fs.azure.sas.fixed.token", "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-09-30T00:03:45Z&st=2025-09-08T15:48:45Z&spr=https&sig=cny38M%2FKcd5Up3ev6zuq3sOG06XyG%2BfugTeakKjs%2BlE%3D") \
    .load(sourcesilverCSVFile)

In [0]:
display(sourceCSV_silverDF)

In [0]:
from pyspark.sql.functions import sum, col, count

gold_daily = (sourceCSV_silverDF
    .groupBy("Published_at")
    .agg(
        sum("Views").alias("total_views"),
        sum("Likes").alias("total_likes"),
        sum("Comments").alias("total_comments"),
        count("VideoID").alias("videos_uploaded")
    )
    .orderBy("Published_at")
)



In [0]:
gold_daily.show() #-Only Showing Top 20 Rows

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc, col

# Window across all rows ordered by likes
windowSpec = Window.orderBy(desc("total_likes"))

# Apply row_number and filter top 3
top3_videos = gold_daily.withColumn(
    "rank", row_number().over(windowSpec)
).filter(col("rank") <= 3) \
 .select("Published_at", "total_views", "total_likes", "total_comments", "videos_uploaded", "rank")

top3_videos.show()



In [0]:
from pyspark.sql.functions import current_timestamp, date_format
#ADD LOAD DATE COLUN
from pyspark.sql.functions import current_date, date_format

# generate load_date as real DATE
top3_videos = top3_videos.withColumn("load_date", current_date()).withColumn("load_date", date_format("load_date", "yyyy-MM-dd"))


In [0]:
display(top3_videos)

In [0]:
top3_videos.write.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("fs.azure.account.auth.type", "SAS") \
    .option("fs.azure.sas.token.provider.type", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider") \
    .option("fs.azure.sas.fixed.token", "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-09-30T00:03:45Z&st=2025-09-08T15:48:45Z&spr=https&sig=cny38M%2FKcd5Up3ev6zuq3sOG06XyG%2BfugTeakKjs%2BlE%3D") \
    .mode("overwrite").save(Target_GoldFile)